### DC3 Algorithm

In [1]:
import radix_DC3_tuples as rd
import numpy as np

[([0, 0, 0], 5), ([97, 98, 0], 7), ([98, 99, 97], 3), ([98, 99, 97], 4), ([99, 97, 98], 2), ([99, 97, 98], 1), ([99, 97, 99], 6)]
[([0, 0], 4), ([97, 98], 7), ([98, 99], 1), ([98, 99], 2), ([99, 97], 3), ([99, 97], 5), ([99, 97], 6)]


In [2]:
#Import test

lis_test=[([98,99,97],3),([98,99,97],4),([99,97,98],2),([0,0,0],5),([99,97,98],1),([99,97,99],6),([97,98,0],7)]
rd.radix_sort(lis_test,3)
print(lis_test)
lis_test=[([98,99],1),([98,99],2),([99,97],3),([0,0],4),([99,97],5),([99,97],6),([97,98],7)]
rd.radix_sort(lis_test,2)
print(lis_test)

[([0, 0, 0], 5), ([97, 98, 0], 7), ([98, 99, 97], 3), ([98, 99, 97], 4), ([99, 97, 98], 2), ([99, 97, 98], 1), ([99, 97, 99], 6)]
[([0, 0], 4), ([97, 98], 7), ([98, 99], 1), ([98, 99], 2), ([99, 97], 3), ([99, 97], 5), ([99, 97], 6)]


In [16]:
def DC3(S, P_12_base = []) :
    """
    Create the suffix array from DC3 algorithm, could be recursive if needed
    
    Args:
        S (str): string
        P_12_base : store P1+2 from recursion to map correctly recursivity
    
    Return:
        index_012 : suffix array of S
        order_12 : order of the next recursion to map correctly recursivity
    """
    
    DC3_table = np.zeros((3,len(S) + 3), dtype=int) # Les caractères sentinelles sont déja là !
    """
    Table de DC3 qui contient en chaque ligne : 
    Ligne 0 : indice du caractère
    Ligne 1 : conversion du caractère en nombre
    Ligne 2 : Ordre de l'indice du caractère
    
    """
    for i in range(len(S) + 3) :
            DC3_table[0][i] =  i # On remplace le caractère par son code Ascii
    
    #print(DC3_table)
    
    
    # String conversion : !!!!! à n'executer que lors de la première récursion !!!!!
    if type(S) == str :
        S_l = [*S] # On sépare caractère par caractère : "ATGC" devient ["A","T","G","C"]
        for i in range(len(S_l)) :
            DC3_table[1][i] =  ord(S_l[i]) # On rempli le caractère par son code Ascii dans la table
    else :
         for i in range(len(S)) :
            DC3_table[1][i] =  S[i] # Cas où l'on rentre dans la boucle une deuxième fois ou plus, pas de conversion
    
    # Cas où la chaine n'est composé que d'une seule lettre : trivial car DC3 = ordre des indices en décroissant
    if (sum(DC3_table[1])/len(S) == DC3_table[1][0]) : # On teste si la chaine n'est composé que d'une seule lettre
        return [*range(len(S)-1,-1,-1)]
    
    
    """
    for i in range(len(S), len(S) + 3, 1) :
        conversion_l.append(0) # Ajout des 3 caractères sentinelles # A garder si pas tables de 0
    """
    
    # On crée P0, P1, P2 et P1+P2 :
    
    P0 = [*range(0,len(S)+1,3)] 
    P1 = [*range(1,len(S)+1,3)]
    P2 = [*range(2,len(S)+1,3)]
    
    P_12 = P1 + P2
    #print(P0, P1, P2)
    #print(P_12)
    
    #Obtention des triplets à partir de P1+P2 :
    
    R_12 = []
    for val in P_12 :
        R_12.append([list(DC3_table[1][val:val+3]), val])

    #print(R_12)
    
    rd.radix_sort(R_12,3) # On trie les triplets
    
    #print(R_12)
    
    index_12 = [] # Liste des indexes de R12 trié
    order_count = 1 # Compteur pour remplir l'ordre
    recur = False # Etat de la récursion tourné True si on a des égalités d'ordre
    for j in range(len(R_12)) : # On parcours tous les triplets triés
        index_12.append(R_12[j][1]) # ... pour lui attribuer son index depuis P_12
        DC3_table[2][R_12[j][1]] = order_count # Et on ajoute l'ordre dans la table
        if j < len(R_12)-1 :
            if R_12[j][0] != R_12[j+1][0] : # On teste l'égalité des triplets pour mettre l'ordre
                order_count += 1
            else :
                recur = True # On a égalité, donc on doit relancer l'algorithme à la fin des for
        else :
            order_count += 1
    
    #print(DC3_table, index_12)
    
    if recur == True :
        new_S = [] # On crée T' la séquence des orders suivant l'ordre de P12
        for l in P_12 :
            new_S.append(DC3_table[2][l])
        #print(new_S)
        index_012 = DC3(new_S, P_12) # On doit récupérer ces deux paramètres sinon ça marche pas
        index_12 = []
        for ind,val in index_012 :
            DC3_table[2][ind] = val
            index_12.append(ind)
            
        #print(DC3_table)
    
    R_0 = [] # On crée la dernière partie à trier
    for val in P0 :
        R_0.append([[int(DC3_table[1][val]), DC3_table[2][val + 1]], val]) # On crée R0 avec son indice
        
    #print(R_0)
    
    rd.radix_sort(R_0,2)
    
    #print(R_0)
    
    index_0 = [] # Liste des indexes de R0 trié
    for k in range(len(R_0)) : # On parcours tous les doublets triés
       index_0.append(R_0[k][1]) # On récupère l'indice
    
    print(DC3_table[0:2], index_0, index_12)
    
    index_012 = [] # On crée l'index final en ordonant 0 et 1,2
    i_0 = 0
    i_12 = 0
    while (i_0 < len(index_0) or i_12 < len(index_12)) : # On prends tout les éléments : on vide index 0 et 12
        if i_0 == len(index_0) : # Cas où index 0 est vide, on rempli avec index 12
            index_012.append(index_12[i_12])
            i_12 += 1
        elif i_12 == len(index_12) : # Cas où index 12 est vide, on rempli avec index 0
            index_012.append(index_0[i_0])
            i_0 += 1
        elif int(DC3_table[1][index_0[i_0]]) > int(DC3_table[1][index_12[i_12]]) : # Cas où index 12 arrive avant index 0
            index_012.append(index_12[i_12])
            i_12 += 1
        elif int(DC3_table[1][index_0[i_0]]) < int(DC3_table[1][index_12[i_12]]) : # Cas où index 0 arrive avant index 12
            index_012.append(index_0[i_0])
            i_0 += 1
        else : # Cas d'égalité sur l'indice : si les 2 indexes renvoient le même nombre 
            if index_12[i_12] % 3 == 1 :
                if index_12.index(index_0[i_0] + 1) > index_12.index(index_12[i_12] + 1) : # Cas où index 12 au deuxième terme arrive avant index 0 au deuxième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au deuxième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
            else :
                if int(DC3_table[1][index_0[i_0] + 1]) > int(DC3_table[1][index_12[i_12] + 1]) : # On teste dabord cas où index 12 + 1 arrive avant index 0 + 1
                    index_012.append(index_12[i_12])
                    i_12 += 1
                elif int(DC3_table[1][index_0[i_0] + 2]) < int(DC3_table[1][index_12[i_12] + 2]) : # Cas où index 0 + 1 arrive avant index 12 + 1
                    index_012.append(index_0[i_0])
                    i_0 += 1
                elif index_12.index(index_0[i_0] + 2) > index_12.index(index_12[i_12] + 2) : # Cas où index 12 au deuxième terme arrive avant index 0 au troisième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au troisième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
    
    #print(index_012)
    
    if int(DC3_table[1][index_012[0]]) == 0 : # On enlève le terme sentinel s'il est présent
        index_012 = index_012[1:]
        
    #print(index_012)
    
    if len(P_12_base) > 0 : # Mapping sur recursion -1 si existe
        new_index_012 = []
        for n in range(len(index_012)) :
            new_index_012.append([P_12_base[index_012[n]], n])
        #print(new_index_012)
        index_012 = new_index_012
   
    

    return index_012 # Retourne le suffix array si dernière récursion


In [17]:
DC3_test1 = DC3("ATGCGGCTTACGTAG")

print(DC3_test1)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
 [65 84 71 67 71 71 67 84 84 65 67 71 84 65 71  0  0  0]] [15, 9, 0, 3, 6, 12] [13, 10, 14, 2, 5, 4, 11, 8, 1, 7]
[9, 13, 0, 3, 10, 6, 14, 2, 5, 4, 11, 8, 12, 1, 7]


In [18]:
print(DC3("abcabcacab"))

[[0 1 2 3 4 5 6 7 8 9]
 [3 3 4 1 4 5 2 0 0 0]] [3, 6, 0] [7, 1, 2, 4, 5]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [97 98 99 97 98 99 97 99 97 98  0  0  0]] [0, 3, 6, 9] [10, 8, 1, 4, 7, 2, 5]
[8, 0, 3, 6, 9, 1, 4, 7, 2, 5]


In [19]:
print(DC3("AAAAAAAA"))

[7, 6, 5, 4, 3, 2, 1, 0]


In [20]:
print(DC3("TTTCCAATTAATTATCAAGTCTGTTTTCCAATTAATTATCAAGTCTGTTTTGGGTTTCCAATTAATTATCAAGTTTCCAATTAATTATCAAGTCTGTTTTGGGTTTCCAATTAATTATCAAGTCTGTTTTGGGACTCTGCATCTGTTTTGGGACTCTGCATTTGGGTTTCCAATTAATTATCAAGTCTGTTTTGGGACTCTGCA"))

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
  48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
 [ 9  4 11 43 27 40  7 13 36 28 46 15 18 52 30 44 12 47 14 34 21 16  8 20
  56 49 25 51 19  3  1 29 53 16 54 24 23 26 31 48 32  4 10 42 15 17  2 38
   5 55 41 22 33 35  6 37 12 45 39 25 50  0  0  0]] [30, 48, 54, 6, 0, 42, 18, 21, 33, 45, 12, 51, 36, 9, 3, 15, 57, 39, 60, 27, 24] [61, 46, 29, 41, 1, 22, 2, 56, 16, 7, 44, 11, 28, 23, 20, 35, 59, 26, 37, 4, 31, 14, 38, 40, 52, 19, 53, 8, 55, 47, 58, 5, 50, 43, 10, 17, 25, 13, 32, 34, 49]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
  48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
  72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93]
 [16 10 30 41  2 45 16 11 18 33 40 46  5 28 26 17 39 25 46  5 28 26 14